In [ ]:
import tkinter as tk
from tkinter import scrolledtext, messagebox
from bs4 import BeautifulSoup
import re
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import nltk
import numpy as np
import pandas as pd
import spacy
from transformers import BertTokenizer, BertForTokenClassification
import torch

# Load the IMDB dataset
data = pd.read_csv('Sentiment Dataset.csv')

# Load the training data from the IMDB dataset
train_reviews = data['review'][:40000]
train_sentiments = data['sentiment'][:40000]

# Load and fit the TF-IDF vectorizer on the training data
vectorizer = TfidfVectorizer(min_df=0, max_df=1, use_idf=True, ngram_range=(1, 3))
vectorizer.fit(train_reviews)

# Transform the training reviews using the fitted vectorizer
train_reviews_transformed = vectorizer.transform(train_reviews)

# Load the trained model and label binarizer
# Note: Make sure to have these objects saved from your training phase
model = LogisticRegression(penalty='l2', max_iter=500, C=1, random_state=42)
label_binarizer = None  # LabelBinarizer is not needed for logistic regression

# Fit the Logistic Regression model with the training data
model.fit(train_reviews_transformed, train_sentiments)

# Load spaCy NER model
nlp = spacy.load("en_core_web_sm")

# Load BERT tokenizer and model for NER
tokenizer_bert = BertTokenizer.from_pretrained('bert-base-uncased')
#model_bert = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(nlp.entity.labels_))
model_bert = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=30)


# Create the main window
app = tk.Tk()
app.title("Sentiment Chatbot for Kids")

# Set a custom icon (replace 'your_logo.png' with the path to your logo file)
logo_image = tk.PhotoImage(file='icon.png')  # Replace with your logo file
app.iconphoto(True, logo_image)

# Create chat window
chat_window = scrolledtext.ScrolledText(app, wrap=tk.WORD, width=40, height=15, state=tk.DISABLED)
chat_window.pack(padx=10, pady=10)

# Create input field
entry = tk.Text(app, wrap=tk.WORD, width=40, height=3)
entry.pack(padx=10, pady=10)

# Instantiate the tokenizer
tokenizer = ToktokTokenizer()

# Function for preprocessing user input
def preprocess_input(text):
    # Function to remove HTML tags
    def strip_html(text):
        soup = BeautifulSoup(text, "html.parser")
        return soup.get_text()

    # Function to remove special characters
    def remove_special_characters(text, remove_digits=True):
        pattern = r'[^a-zA-z\s]'
        text = re.sub(pattern, '', text)
        return text

    # Function to remove stopwords
    def remove_stopwords(text):
        stopword_list = set(nltk.corpus.stopwords.words('english'))
        tokens = tokenizer.tokenize(text)
        tokens = [token.strip() for token in tokens]
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
        filtered_text = ' '.join(filtered_tokens)
        return filtered_text

    # Apply preprocessing steps
    text = strip_html(text)
    text = remove_special_characters(text)
    text = remove_stopwords(text)
    return text

# Function for sentiment prediction
def predict_sentiment(model, vectorizer, user_input):
    # Preprocess user input
    preprocessed_input = preprocess_input(user_input)
    
    # Vectorize the preprocessed input
    input_vectorized = vectorizer.transform([preprocessed_input])
    
    # Predict sentiment
    prediction = model.predict(input_vectorized)
    return prediction[0]

# Function for Named Entity Recognition (NER)
def extract_entities(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities



# Flag to track whether the farewell message has been displayed
farewell_displayed = False

# Function to handle user input and display sentiment
def handle_input():
    global farewell_displayed
    user_input = entry.get("1.0", tk.END).strip()
    if user_input:
        try:
            # Check for greetings in user input
            greetings = ["hello", "hi", "hey", "greetings", "hi there", "hallo", "good day", "good morning"]
            if any(greeting in user_input.lower() for greeting in greetings):
                # Reply with a friendly greeting
                bot_reply = "Hello! Little Angel! What have you heard today?"
                farewell_displayed = False  # Reset the farewell_displayed flag

            # Check for farewell in user input
            elif any("bye" in user_input.lower() for farewell in ["bye", "goodbye", "see you", "see ya", "tschüss :p"]):
                # Reply with farewell only if not displayed before
                if not farewell_displayed:
                    bot_reply = "Goodbye!"
                    
                    # Display the farewell message
                    chat_window.config(state=tk.NORMAL)
                    chat_window.insert(tk.END, f"You: {user_input}\nBot: {bot_reply}\n\n")
                    chat_window.config(state=tk.DISABLED)

                    farewell_displayed = True  # Set the farewell_displayed flag

                    # Close the window after a delay
                    app.after(3000, app.destroy)
                else:
                    bot_reply = ""  # If farewell already displayed, set an empty reply

            else:
                # Extract named entities
                entities = extract_entities(user_input)

                # Predict sentiment
                prediction = predict_sentiment(model, vectorizer, user_input)
                sentiment_label = "Positive" if prediction == 1 else "Negative"

                # Construct the bot's reply based on the prediction
                if sentiment_label == "Negative":
                    bot_reply = f"The word is {sentiment_label} please don't use it with others"
                else:
                    bot_reply = f"The word is {sentiment_label} please feel free to use it with others "

            # Display the conversation in the chat window
            chat_window.config(state=tk.NORMAL)
            chat_window.insert(tk.END, f"You: {user_input}\nBot: {bot_reply}\n\n")
            chat_window.config(state=tk.DISABLED)

            # Clear the input field
            entry.delete("1.0", tk.END)

        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {str(e)}")


# Create submit button
submit_button = tk.Button(app, text="Ask me", command=handle_input)
submit_button.pack(padx=10, pady=10)

# Run the application
app.mainloop()
